In [2]:
import requests
import xml.etree.ElementTree as ET
import csv
from datetime import datetime, timedelta

# API Credentials
username = 'alean@bu.edu'
password = 'Mq75eg8pxTBCEKY'

# API Base URL
base_url = "https://webservices.iso-ne.com/api/v1.1//hourlyloadforecast/day/"

# Function to query API and fetch data
def fetch_data(date):
    url = f"{base_url}{date}"
    print(f"Requesting URL: {url}")
    response = requests.get(url, auth=(username, password))
    if response.status_code == 200:
        return response.text
    else:
        print(f"Error fetching data for {date}: {response.status_code}, {response.text}")
        return None

# Function to parse XML and save to CSV
def parse_and_save_to_csv(xml_data, csv_file):
    # Parse the XML data
    root = ET.fromstring(xml_data)

    # Extract the namespace from the XML
    namespace = {'ns': 'http://WEBSERV.iso-ne.com'}

    # Find all HourlyLoadForecast elements
    forecast_entries = root.findall('ns:HourlyLoadForecast', namespace)

    # Open the CSV file for writing
    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Write the header if the file is empty
        if file.tell() == 0:
            writer.writerow(["BeginDate", "LoadMw"])

        # Write each forecast entry to the CSV
        for entry in forecast_entries:
            begin_date = safe_find_text(entry, 'ns:BeginDate', namespace)
            load_mw = safe_find_text(entry, 'ns:LoadMw', namespace)

            # Skip rows with missing required data
            if begin_date is None or load_mw is None:
                print(f"Skipping entry due to missing data: {entry}")
                continue

            writer.writerow([begin_date, load_mw])

# Helper function to safely extract text
def safe_find_text(element, tag, namespace):
    found = element.find(tag, namespace)
    return found.text if found is not None else None

# Main function to iterate over dates and process data
def main(start_date, end_date, csv_file):
    current_date = start_date
    while current_date <= end_date:
        formatted_date = current_date.strftime("%Y%m%d")
        print(f"Fetching data for {formatted_date}...")
        xml_data = fetch_data(formatted_date)
        if xml_data:
            parse_and_save_to_csv(xml_data, csv_file)
        current_date += timedelta(days=1)
    print(f"Data saved to {csv_file}")

# Specify date range and output CSV file
start_date = datetime(2024, 12, 3)  # Start date
end_date = datetime(2024, 12, 4)    # End date
csv_file = "hourly_load_forecast.csv"

# Run the script
main(start_date, end_date, csv_file)


Fetching data for 20241203...
Requesting URL: https://webservices.iso-ne.com/api/v1.1//hourlyloadforecast/day/20241203
Fetching data for 20241204...
Requesting URL: https://webservices.iso-ne.com/api/v1.1//hourlyloadforecast/day/20241204
Data saved to hourly_load_forecast.csv
